In [6]:
from PIL import Image, ImageGrab
from IPython.display import IFrame
import win32api
import numpy as np
from itertools import product
import time
import cv2
import win32.lib.win32con as win32con

%run CandyRecognition.ipynb
%run Solver.ipynb
cand_rec= CandyRecog()
cand_rec.train()
solv= Solver()

In [7]:
url = "https://www.cooljuegos.com/juego-en-linea/candy-crush/"
game_board = np.zeros((9, 9), dtype=np.int32)

bbox= [267, 100, 1860, 1504]
ax_x= (bbox[2]-bbox[0])//9
ax_y= (bbox[3]-bbox[1])//9

match_list = [(0, 1, 13, 19), (2, 3, 14, 20), (4, 5, 15, 21), (6, 7, 18, 22), (8, 9, 16, 23), (10, 11, 17, 24)]
special_candies = [1, 3, 5, 7, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
simple_candies = [0, 2, 4, 6, 8, 10]
striped_candies_h = [1, 3, 5, 7, 9, 11]
striped_candies_v = range(13, 19)
striped_candies = striped_candies_h[:]
striped_candies.extend(striped_candies_v)
wrapped_candies = range(19, 25)
chocolate = [12]

board_dict = {0: 'blue       ', 1: 's_h_blue   ', 2: 'green      ', 3: 's_h_green  ', 4: 'orange     ', 5: 's_h_orange ',
              6: 'purple     ', 7: 's_h_purple ', 8: 'red        ', 9: 's_h_red    ', 10: 'yellow     ', 11: 's_h_yellow ',
              12: 'chocolate  ', 13: 's_v_blue   ', 14: 's_v_green  ', 15: 's_v_orange ', 16: 's_v_red    ',
              17: 's_v_yellow ', 18: 's_v_purple ', 19: 'blue_wrappe', 20: 'green_wrapp', 21: 'orange_wrap',
              22: 'purple_wrap', 23: 'red_wrapped', 24: 'yellow_wrap', -1: 'empty     '}

ref_img= None
tsem= 0.05

img_end_game = Image.open('end_screen.jpg')
img_end_game = img_end_game.resize((int(img_end_game.size[0]/4), int(img_end_game.size[1]/4)), Image.NEAREST)

In [8]:
def get_desktop_coords(cell):
    x = int((bbox[0] + cell[1] * ax_x + ax_x/2)/2.5)
    y = int((bbox[1] + cell[0] * ax_y + ax_y/2)/2.5)
    return x, y

def do_move(move):
    start, end = move
    start_w = get_desktop_coords(start)
    end_w = get_desktop_coords(end)
    win32api.SetCursorPos(start_w)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, start_w[0], start_w[1], 0, 0)
    time.sleep(tsem)
    win32api.SetCursorPos(end_w)
    time.sleep(tsem)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, end_w[0], end_w[1], 0, 0)

def ScreenShotBoard():
    im = ImageGrab.grab(bbox, all_screens=True)
    #im.save("screen.jpg")
    #(1215/9= 135, 952/8= 119)
    #im2= Image.open("screen.jpg")
    for y in range(9):
        for x in range(9):
            candy_box = (x*ax_x, y*ax_y, (x+1)*ax_x, (y+1)*ax_y)
            candy = im.crop(candy_box)
            #candy.save('Candies/{0}_{1}.bmp'.format(y, x))
            resized_img = candy.resize((45, 45), Image.BILINEAR)
            np_img = np.array(resized_img.getdata()).flatten()
            game_board[y][x] = cand_rec.predict([np_img])
    return(im)

def print_board(board):
    for line in board:
        for elem in line:
            print(board_dict[elem] + ' ',end= "")
        print()

def board_is_moving():
    global ref_img
    img = ImageGrab.grab()
    img = img.crop(bbox)
    has_movement = True
    if ref_img:
        has_movement = DiffImages(img, ref_img) < 95
    ref_img = img
    return (has_movement)

def DiffImages(im1, im2):
    opencvImage1 = cv2.cvtColor(np.array(im1), cv2.COLOR_RGB2BGR)
    img1 = cv2.cvtColor(opencvImage1, cv2.COLOR_BGR2GRAY)
    opencvImage2 = cv2.cvtColor(np.array(im2), cv2.COLOR_RGB2BGR)
    img2 = cv2.cvtColor(opencvImage2, cv2.COLOR_BGR2GRAY)
    diff = cv2.subtract(img1, img2)
    err = np.sum(diff**2)
    mse = err/(float(img1.shape[0]*img1.shape[1]))
    
    return(100-mse)

In [9]:
def main():
    time.sleep(1)
    total_moves = 0
    while True:
        if not board_is_moving():
            board_img = ScreenShotBoard()
            board_img = board_img.resize((int(board_img.size[0]/4), int(board_img.size[1]/4)), Image.NEAREST)
            if DiffImages(board_img, img_end_game)== 100:
                break
            move = solv.solve_board(game_board)
            do_move(move)
            total_moves += 1
        time.sleep(tsem)
    print('Total moves done: ' + str(total_moves))

In [12]:
main()

Total moves done: 240


In [8]:
IFrame(url, width=900, height=900)

In [ ]:
'''
tsem (timesleepmouse)
tsep (timesleepprogram)


tsem= 0.4 tsep= 0.7 [42820, 79600, 37510][1]
tsem= 0.5 tsep= 0.8 [43150, 35600] [1,1 ]
tsem= 0.2 tsep= 0.5 [, ] [, ]
'''